## 9. Testing and Debugging

### 80 Consider Interactive Debugging with `pdb`

```python
# always_breakpoint.py

import math

def compute_rmse(observed, ideal):
    total_err_2 = 0
    count = 0
    for got, wanted in zip(observed, ideal):
        err_2 = (got - wanted) ** 2
        breakpoint()  # Start the debugger here
        total_err_2 += err_2
        count += 1

    mean_err = total_err_2 / count
    rmse = math.sqrt(mean_err)
    return rmse

result = compute_rmse(
    [1.8, 1.7, 3.2, 6],
    [2, 1.5, 3, 5])
print(result)
```

```shell
$ python always_breakpoint.py
> /home/jay/_doc/spike/python/effective/80/always_breakpoint.py(25)compute_rmse()
-> total_err_2 += err_2
(Pdb) where
  /home/jay/_doc/spike/python/effective/80/always_breakpoint.py(32)<module>()
-> result = compute_rmse(
> /home/jay/_doc/spike/python/effective/80/always_breakpoint.py(25)compute_rmse()
-> total_err_2 += err_2
(Pdb) 
```

```
where
u, up
d, down

step     -- step in
next     -- step over
return   -- step out
continue
quit
```

```python
# conditional_breakpoint.py

import math

def compute_rmse(observed, ideal):
    total_err_2 = 0
    count = 0
    for got, wanted in zip(observed, ideal):
        err_2 = (got - wanted) ** 2
        if err_2 >= 1:  # Start the debugger if True
            breakpoint()
        total_err_2 += err_2
        count += 1
    mean_err = total_err_2 / count
    rmse = math.sqrt(mean_err)
    return rmse

result = compute_rmse(
    [1.8, 1.7, 3.2, 7],
    [2, 1.5, 3, 5])
print(result)
```

```shell
$ python conditional_breakpoint.py
> /home/jay/_doc/spike/python/effective/80/conditional_breakpoint.py(25)compute_rmse()
-> total_err_2 += err_2
(Pdb) wanted
5
(Pdb) got
7
(Pdb) err_2
4
(Pdb) continue
1.014889156509222
```

```python
# postmortem_breakpoint.py

import math

def compute_rmse(observed, ideal):
    total_err_2 = 0
    count = 0
    for got, wanted in zip(observed, ideal):
        err_2 = (got - wanted) ** 2
        total_err_2 += err_2
        count += 1

    mean_err = total_err_2 / count
    rmse = math.sqrt(mean_err)
    return rmse

result = compute_rmse(
    [1.8, 1.7, 3.2, 7j],  # Bad input
    [2, 1.5, 3, 5])
print(result)
```

```shell
$ python postmortem_breakpoint.py 
Traceback (most recent call last):
  File "postmortem_breakpoint.py", line 31, in <module>
    result = compute_rmse(
  File "postmortem_breakpoint.py", line 28, in compute_rmse
    rmse = math.sqrt(mean_err)
TypeError: can't convert complex to float
```

```shell
$ python -m pdb -c continue postmortem_breakpoint.py 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/pdb.py", line 1704, in main
    pdb._runscript(mainpyfile)
  File "/usr/local/lib/python3.8/pdb.py", line 1573, in _runscript
    self.run(statement)
  File "/usr/local/lib/python3.8/bdb.py", line 580, in run
    exec(cmd, globals, locals)
  File "<string>", line 1, in <module>
  File "/home/jay/_doc/spike/python/effective/80/postmortem_breakpoint.py", line 17, in <module>
    import math
  File "/home/jay/_doc/spike/python/effective/80/postmortem_breakpoint.py", line 28, in compute_rmse
    rmse = math.sqrt(mean_err)
TypeError: can't convert complex to float
Uncaught exception. Entering post mortem debugging
Running 'cont' or 'step' will restart the program
> /home/jay/_doc/spike/python/effective/80/postmortem_breakpoint.py(28)compute_rmse()
-> rmse = math.sqrt(mean_err)
(Pdb) 
```

```python
# my_module.py

import math

def squared_error(point, mean):
    err = point - mean
    return err ** 2

def compute_variance(data):
    mean = sum(data) / len(data)
    err_2_sum = sum(squared_error(x, mean) for x in data)
    variance = err_2_sum / (len(data) - 1)
    return variance

def compute_stddev(data):
    variance = compute_variance(data)
    return math.sqrt(variance)
```

```shell
$ python
>>> import my_module
>>> my_module.compute_stddev([5])
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/home/jay/_doc/spike/python/effective/80/my_module.py", line 30, in compute_stddev
    variance = compute_variance(data)
  File "/home/jay/_doc/spike/python/effective/80/my_module.py", line 26, in compute_variance
    variance = err_2_sum / (len(data) - 1)
ZeroDivisionError: float division by zero
>>> import pdb; pdb.pm()
> /home/jay/_doc/spike/python/effective/80/my_module.py(26)compute_variance()
-> variance = err_2_sum / (len(data) - 1)
(Pdb) err_2_sum
0.0
(Pdb) len(data)
1
(Pdb) 
```

> - 프로그램에서 관심이 있는 부분에 `breakpoint` 내장 함수 호출을 추가하면 (프로그램을 실행하던 중에) 그 위치에서 파이썬 대화형 디버거를 시작할 수 있다.
> - 파이썬 디버거 프롬프트는 완전한 파이썬 셸이기 때문에 실행 중인 프로그램의 상태를 원하는 대로 관찰하거나 변경할 수 있다.
> - `pdb3` 셸 명렬어를 사용하면 프로그램 실행을 정밀하게 제어할 수 있고, 프로그램의 상태를 관찰하는 과정과 프로그램을 진행시키는 과정을 번갈아가며 수행할 수 있다.
> - 독립 실행한 파이썬 프로그램에서 예외가 발생하는 경우, `pdb` 모듈을 사용(`python -m pdb -c continue /path/to/program`)하거나 대화형 파이썬 인터프리터(`import pdb; pdb.pm()`)를 사용해 디버깅할 수 있다.